 0. Import the relevant packages

In [174]:
import pandas as pd
import os

# Creating the choice set
From   
![image.png](https://cdn.jamanetwork.com/ama/content_public/journal/psych/938422/m_yoi190087ea.png?Expires=1691689740&Signature=OvJ06Ayrw5fJLEycmRUj9I6GyITQwp89ZN5rVQDOm8dU1z6ZvhwNCwFV8yKnBDDEgxFqnaUQWgV7viAKHn5NpRdiVPo-Rgc8kjwYTn-0dnbudBGSoeVtaw8gTSIUiLLnWWVj2sPafVJA9NhK3bivpgCt2RLxI2UJRYUuNUmrrx1CFwyJTKYPlljeVSSEZ3Iwc9dNmvZ4p71MmTQSt8a9KcXSmZ0r0IZhQ187TDnqEFIiAN3mv9A7snMUAnhROj0Fg1p5COdoTVA9Z3RVNw8AT8QgJNSKl44244OjAauQwcnETbDdh2NPeCv4NGXYVclFTopaMriRPqUO4y-QPdDQjw__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA)

We now have all the elements to calculate subjective value.


In [216]:
''' Input should have this form

cols = ['crdm_lott_amt', 'crdm_lott_p', 'crdm_amb_lev', 'alpha', 'beta', 'gamma']

df_indif = pd.DataFrame([[20, 50, 0, 0.495323857, 0.495323857, 4.100226383]], columns= cols)
print(df_indif)
filename = 'csv/example_input.csv'
df_indif.to_csv(filename)

'''
subject_SE = 'csv/subject_input.csv'
df = pd.read_csv(subject_SE) 
df

,Unnamed: 0,crdm_lott_amt,crdm_lott_p,crdm_amb_lev,alpha,beta,gamma
0,0,20.0,50.0,0.0,0.495324,0.22168,4.100226


In [217]:
#Declare the variables
p = df['crdm_lott_p'].values[0]/100  #This is the probability of reward
v = df['crdm_lott_amt'].values[0] #The amount to be gained
alpha = df['alpha'].values[0] #Participant's or ADO alpha parameter
beta = df['beta'].values[0] #Participant's or ADO beta parameter
A= df['crdm_amb_lev'].values[0] #Ambiguity level

#SV calculation and our probabilities options
sv = (p - beta*A/2)*v**alpha  
p_options = [0.87, 0.75, 0.62, 0.5, 0.38, 0.25, 0.13]

#Confirme the SE trial structure
print ('The probability of reward: ' + str(p))
print ('The amount to be earned: ' + str(v))
print ('The alpha parameter from the model: ' + str(alpha))
print ('The beta parameter from the model: ' + str(beta))
print ('The ambiguity level of the SE trial: ' + str(A))
print('The SV of the equivalence trial is ' + str(sv))


The probability of reward: 0.5
The amount to be earned: 20.0
The alpha parameter from the model: 0.495323857
The beta parameter from the model: 0.221680178
The ambiguity level of the SE trial: 0.0
The SV of the equivalence trial is 2.204962459791904


From the SV, we compute V, using the formula:

# $v = \left(\frac{SV}{p-\frac{\beta A}{2}}\right)^\frac{1}{\alpha}$


# 2. Create the choice set. 
- We want to make 3 trials around the point of subjective equality. 

- We want 2 extreme trials. One at $50 and one at the -SV of the 50\$ trial for each probability option. 

- We want 4 intermediate trials.  


In [218]:
#First, we compute the Subjective indiference trials.

def SE_options(sv, alpha, beta, A): #This function take as parameters SV, alpha, beta and ambiguity level
  valuesP = {} #The output will be an empty dictionary with the probabilities as keys and rewards as values.
  for p in p_options: #A loop to iterate through all the probabilities optins.
    v_new = (sv / (p-beta*A/2)) ** (1 / alpha)
    valuesP[p] = v_new #for every p it includes the calculated value
  return valuesP


new_choicesp = SE_options(sv, alpha,beta,A)
SE_trials = {k: round(v, 2) for k, v in new_choicesp.items()}
print('Trials with the same subjective value for all of our Prob options: \n' + str(SE_trials))


Trials with the same subjective value for all of our Prob options: 
{0.87: 6.54, 0.75: 8.82, 0.62: 12.95, 0.5: 20.0, 0.38: 34.81, 0.25: 81.05, 0.13: 303.48}


- Next, we create new trials by adding and substracting \$2 from the SE trials values.

In [219]:
#Centered around subjective indifference +/- and $2
def centered ():
  cent_trials1 = {}
  for k,v1 in SE_trials.items():
    cent_trials1[k] = [float(v1) - 2.0,float(v1) + 2.0]
  return cent_trials1

cent_trials1 = centered()
print("Trials centered + - $2 around SE \n" + str(cent_trials1))

Trials centered + - $2 around SE 
{0.87: [4.54, 8.54], 0.75: [6.82, 10.82], 0.62: [10.95, 14.95], 0.5: [18.0, 22.0], 0.38: [32.81, 36.81], 0.25: [79.05, 83.05], 0.13: [301.48, 305.48]}


In [220]:
#Trials around SE (including the SE)
cent_trials = {}
for prob in SE_trials:
    value1 = float(SE_trials[prob]) #We convert the values in SE_trials into floats
    value2 = cent_trials1[prob] #We call the values from the cent_trials 1 dict. This have the plus/minus $2 but without the SE trials
    cent_trials[prob] = [value1] + value2 #Join the SE trials and the plus/minus $2 tirals
cent_trials
print('Trials around the SE and + - \$2: \n', cent_trials)

Trials around the SE and + - \$2: 
 {0.87: [6.54, 4.54, 8.54], 0.75: [8.82, 6.82, 10.82], 0.62: [12.95, 10.95, 14.95], 0.5: [20.0, 18.0, 22.0], 0.38: [34.81, 32.81, 36.81], 0.25: [81.05, 79.05, 83.05], 0.13: [303.48, 301.48, 305.48]}


In [221]:
# Make it a CSV
# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(cent_trials.items()), columns=['p_reward', 'value_reward'])
df = df.explode('value_reward')
# Specify the filename for the new CSV file
cent_trialsCSV = 'csv/cent_trials.csv'

# Save the DataFrame to a CSV file
df.to_csv(cent_trialsCSV, index=False)

print(f"CSV file '{cent_trialsCSV}' created successfully.")
pd.read_csv(cent_trialsCSV)

CSV file 'csv/cent_trials.csv' created successfully.


,p_reward,value_reward
0,0.87,6.54
1,0.87,4.54
2,0.87,8.54
3,0.75,8.82
4,0.75,6.82
5,0.75,10.82
6,0.62,12.95
7,0.62,10.95
8,0.62,14.95
9,0.50,20.00


**Extreme trials:**

To create trials far way from the subjective equality point (SEP). We first make trials with a Value of reward of $50, which is the maximum amount we can pay to participants. Thus this is the highest subjective value we can reach.

In [222]:
extrpos = {} #Create an empty dict
for p in p_options: #Iterate through the probability levels
    extrpos[p] = 50 #Assign them the 50 dollars value
print(extrpos)

{0.87: 50, 0.75: 50, 0.62: 50, 0.5: 50, 0.38: 50, 0.25: 50, 0.13: 50}


In [228]:
#Now, lets calculate the SV of each extreme positive trial
svpos = []
for p in p_options:
    sv2 = (p - beta*A/2)*50**alpha 
    svpos.append(sv2)
svpos
 

[6.040315564667845,
 5.2071685902309,
 4.304592701257544,
 3.4714457268206003,
 2.638298752383656,
 1.7357228634103001,
 0.902575888973356]

In [185]:
#Now, lets calculate the SV of each extreme Negative trial
svevil = []
for p in p_options:
    sv = (p - beta*A/2)*50**alpha 
    negsv = -sv
    svevil.append (negsv)
    print (f'Evil SV for $50 at {p*100}% probability: {svevil}') 
svevil

Evil SV for $50 at 87.0% probability: [-6.040315564667845]
Evil SV for $50 at 75.0% probability: [-6.040315564667845, -5.2071685902309]
Evil SV for $50 at 62.0% probability: [-6.040315564667845, -5.2071685902309, -4.304592701257544]
Evil SV for $50 at 50.0% probability: [-6.040315564667845, -5.2071685902309, -4.304592701257544, -3.4714457268206003]
Evil SV for $50 at 38.0% probability: [-6.040315564667845, -5.2071685902309, -4.304592701257544, -3.4714457268206003, -2.638298752383656]
Evil SV for $50 at 25.0% probability: [-6.040315564667845, -5.2071685902309, -4.304592701257544, -3.4714457268206003, -2.638298752383656, -1.7357228634103001]
Evil SV for $50 at 13.0% probability: [-6.040315564667845, -5.2071685902309, -4.304592701257544, -3.4714457268206003, -2.638298752383656, -1.7357228634103001, -0.902575888973356]


[-6.040315564667845,
 -5.2071685902309,
 -4.304592701257544,
 -3.4714457268206003,
 -2.638298752383656,
 -1.7357228634103001,
 -0.902575888973356]

In [237]:
# Convert the dictionary to a DataFrame
df1 = pd.DataFrame(SE_trials.items(), columns=['p_reward', 'SE_reward'])
#p_reward = df1['p_reward'].values
#value_reward = df1['value_reward'].values
df1['SV_reward'] = (df1['p_reward'] - beta*A/2)*df1['SE_reward']**alpha
df1['SV_safe'] = 5.0**alpha
df1['MaxReward'] = 50
df1['SV_MaxReward'] = svpos
df1['deltaSV'] = df1['SV_MaxReward'] - df1['SV_reward']
df1


,p_reward,SE_reward,SV_reward,SV_safe,MaxReward,SV_MaxReward,deltaSV
0,0.87,6.54,2.205436,2.219303,50,6.040316,3.834880
1,0.75,8.82,2.204826,2.219303,50,5.207169,3.002342
2,0.62,12.95,2.204578,2.219303,50,4.304593,2.100015
3,0.50,20.00,2.204962,2.219303,50,3.471446,1.266483
4,0.38,34.81,2.205090,2.219303,50,2.638299,0.433208
5,0.25,81.05,2.204910,2.219303,50,1.735723,-0.469187
6,0.13,303.48,2.204964,2.219303,50,0.902576,-1.302389


In [186]:
#Now, let's see what Value produces that -SV for all the probability levels
SVevil= -6.040315564667845
def ExtrNeg_options(SVevil, alpha, beta, A): #This function take as parameters SV, alpha, beta and ambiguity level
  NegvaluesP = {} #The output will be an empty dictionary with the probabilities as keys and rewards as values.
  for p in p_options: #A loop to iterate through all the probabilities optins.
    v_new = (SVevil / (p-beta*A/2)) ** (1 / alpha)
    NegvaluesP[p] = v_new #for every p it includes the calculated value
  return NegvaluesP

ExtrNeg_choices = ExtrNeg_options(SVevil, alpha,beta,A)
ExtrNeg1 = {k: '{:.2f}'.format(v) for k, v in new_choicesp.items()}

print('Trials with the least SV for every prob option: \n' + str(ExtrNeg1))


Trials with the least SV for every prob option: 
{0.87: '6.54', 0.75: '8.82', 0.62: '12.95', 0.5: '20.00', 0.38: '34.81', 0.25: '81.05', 0.13: '303.48'}


/var/folders/yx/fkwj2lbx3674djc_4jq5g8vnvntbk4/T/ipykernel_6988/2890011471.py:6: RuntimeWarning: invalid value encountered in scalar power
  v_new = (SVevil / (p-beta*A/2)) ** (1 / alpha)


# 4. The new choices CSV

In [ ]:
#For the extreme positvie trials
# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(extrpos.items()), columns=['p_reward', 'value_reward'])
df = df.explode('value_reward')
 # Unpack the list in 'value_reward' column
# Specify the filename for the new CSV file
extrpos_trialsCSV = 'csv/extrpos.csv'

# Save the DataFrame to a CSV file
df.to_csv(extrpos_trialsCSV, index=False)

print(f"CSV file '{extrpos_trialsCSV}' created successfully.")
pd.read_csv(extrpos_trialsCSV)

In [ ]:
#For the extreme negative trials
# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(extrneg.items()), columns=['p_reward', 'value_reward'])
df = df.explode('value_reward')
 # Unpack the list in 'value_reward' column
# Specify the filename for the new CSV file
extrneg_trialsCSV = 'csv/extrneg.csv'

# Save the DataFrame to a CSV file
df.to_csv(extrneg_trialsCSV, index=False)

print(f"CSV file '{extrneg_trialsCSV}' created successfully.")
pd.read_csv(extrneg_trialsCSV)

In [ ]:
#Final CSV
SE_trialCSV = pd.read_csv(SE_trialsCSV)
cent_trialCSV = pd.read_csv(cent_trialsCSV)
extrpos_trialCSV = pd.read_csv(extrpos_trialsCSV)
extrneg_trialCSV = pd.read_csv(extrneg_trialsCSV)
#Complete trials CSV
crdm_trials = pd.concat([SE_trialCSV,cent_trialCSV, extrpos_trialCSV,extrneg_trialCSV])
crdm_trialsCSV = 'csv/crdm_trials.csv'
crdm_trials.to_csv(crdm_trialsCSV, index = False)
print(f"CSV file concatenated successfully.")
pd.read_csv(crdm_trialsCSV)

In [ ]:
# Complete trials CSV with filtered trials (<$50)
crdm_trials = pd.concat([SE_trialCSV, cent_trialCSV, extrpos_trialCSV, extrneg_trialCSV])

# Convert 'value_reward' column to numeric
crdm_trials['value_reward'] = pd.to_numeric(crdm_trials['value_reward'], errors='coerce')

# Filter rows where 'value_reward' is less than or equal to 50
crdm_50filtered = crdm_trials[crdm_trials['value_reward'] <= 60]

# Specify the filename for the new CSV file with the filtered data
crdm_trials_50cap = 'csv/crdm_trials_50cap.csv'

# Save the filtered DataFrame to a new CSV file
crdm_50filtered.to_csv(crdm_trials_50cap, index=False)

print(f"CSV file filtered successfully.")
pd.read_csv(crdm_trials_50cap)

# To do:
- Add ambiguity to the choice set. 

    -   *My proposal:* randomize some of the trials from the final CSV with different ambiguity levels.

    -  Silvia, is this okay or do you want a more systematic approach to creat the Ambiguous trials choice cet?

In [ ]:
N=10
p = 10*[3]
print(p)

In [ ]:
#Inserting amibuity
import random
df = pd.read_csv(crdm_trials_50cap)

#Define the ambiguity options
crdm_amb_lvl = [24,50,74]


# number of ambiguous trials by type
N_AMB = 10
crdm_amb_opt = [N_AMB*[a] for a in crdm_amb_lvl]
crdm_amb_opt = [item for sublist in crdm_amb_opt for item in sublist]
print(crdm_amb_opt)
random.shuffle(crdm_amb_opt)
print(crdm_amb_opt)



In [ ]:

#Define the number of ambiguous trials by picking X number of random rows.
amb_trials = df.sample(n=len(crdm_amb_opt))

#Assign ambiguity values that match the number of random rows
# amb_levels = [random.choice(crdm_amb_opt) for x in range(15)]
#Assign those values to the random rows
amb_trials['crdm_amb_lev'] = crdm_amb_opt
amb_trials





In [ ]:

crdm_trials['crdm_amb_lev'] = 0.0
# choice_set = crdm_trials
choice_set = pd.concat([crdm_trials,amb_trials],ignore_index=True)

print(choice_set)



In [ ]:
import matplotlib.pyplot as plt

choice_set['SV_lottery'] = (choice_set['p_reward'] - beta*(choice_set['crdm_amb_lev']/100.0/2))*choice_set['value_reward']**alpha
choice_set['SV_safe'] = 5.0**alpha
choice_set['SV_diff'] = choice_set['SV_lottery'] - choice_set['SV_safe']
print(choice_set)

plt.figure(1)
plt.hist(choice_set.loc[choice_set['crdm_amb_lev']==0.0,'SV_diff'],bins=10)

plt.figure(2)
plt.hist(choice_set.loc[choice_set['crdm_amb_lev']>0.0,'SV_diff'],bins=10)



In [ ]:
plt.figure(4)
x = choice_set.loc[choice_set['crdm_amb_lev']==0.0,'value_reward']
y = choice_set.loc[choice_set['crdm_amb_lev']==0.0,'SV_diff']
plt.scatter(x,y)
plt.xlabel('Value of Reward')
plt.ylabel('SV difference (no ambiuguity)')


plt.figure(5)
x = choice_set.loc[choice_set['crdm_amb_lev']>0.0,'crdm_amb_lev']
y = choice_set.loc[choice_set['crdm_amb_lev']>0.0,'SV_diff']
plt.scatter(x,y)
plt.xlabel('Ambiguity level')
plt.ylabel('SV difference (ambiuguity)')



## It's not much but it's honest work
![python](https://plantillasdememes.com/img/plantillas/no-es-mucho-pero-es-trabajo-honesto01569991155.jpg)